This code was only used for the fourier explanation part of the presentation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

In [ ]:
df_bilt = pd.read_csv('Data/de_bilt_weather.csv')
df_bilt2001 = df_bilt.loc[df_bilt['year'] == 2001]

In [ ]:
variables = ['cloud_cover', 'wind_speed', 'wind_gust', 'humidity',   
            'pressure', 'global_radiation', 'precipitation', 'sunshine',
            'temp_mean', 'temp_min', 'temp_max'] 

n_var = len(variables)
n_data = 365

t_array = np.arange(n_data)

In [ ]:
def DFT(data):
    N = len(data)
    
    k_array = np.arange(N)
    t_array = np.arange(N)

    exponentials =  np.exp(-2j * np.pi * np.outer(k_array, t_array) / N)
    coefs = exponentials @ data
    
    return k_array, coefs / N

def reconstruct(t_array, k_array, coefs):
    N = len(coefs)
    data = np.zeros(len(t_array), dtype=object)

    for k in k_array:
        A = np.abs(coefs[k])
        phase = np.angle(coefs[k])
        data += A * np.cos(phase + 2 * np.pi * k * t_array / N)

    return data

In [ ]:
plt.plot(t_array, df_bilt2001['temp_mean'])
plt.xlim(-10, 400)
plt.ylim(-5, 30)
plt.xlabel('t')
plt.ylabel(r'$x_t$')
plt.title('Mean temperature 2009')
plt.savefig('Figures/fourier1.png', dpi=600)

In [ ]:
data = np.array(df_bilt2001['temp_mean'])                
k, coefs = DFT(data)

fig, ax = plt.subplots()
ax.plot(range(n_data), np.abs(coefs))
ax.set_yscale('log')
ax.set(ylim=(10**-3, 100), xlim=(-50, 400), ylabel=r'$|X_k|$', xlabel='k')
ax.set_title('Mean temperature 2009')
plt.savefig('Figures/fourier2.png', dpi=600)

In [ ]:
plt.errorbar(t_array, reconstruct(t_array, [0, 1, 364], coefs), color='tab:red', label='yearly fourier')
plt.errorbar(t_array, reconstruct(t_array, k, coefs), color='black', fmt='-', label='full fourier')
plt.errorbar(t_array, df_bilt2001['temp_mean'], color='tab:blue', fmt='o', markersize=3, label='observed')
plt.xlim(-10, 400)
plt.ylim(-5, 30)
plt.xlabel('t')
plt.ylabel(r'$x_t$')
plt.title('Mean temperature 2009')
plt.legend()
plt.savefig('Figures/fourier3.png', dpi=600)

In [ ]:
coefs_relative = np.abs(coefs) / np.sum(np.abs(coefs))
mask = coefs_relative > 0.005

k_filtered = k[mask].copy()

In [ ]:
plt.errorbar(t_array, reconstruct(t_array, [0, 1, 364], coefs), color='tab:red', label='yearly fourier')
plt.errorbar(t_array, reconstruct(t_array, k_filtered, coefs), color='black', fmt='-', label='partial fourier (>0.5%)')
plt.errorbar(t_array, df_bilt2001['temp_mean'], color='tab:blue', fmt='o', markersize=3, label='observed')
plt.xlim(-10, 400)
plt.ylim(-5, 30)
plt.xlabel('t')
plt.ylabel(r'$x_t$')
plt.title('Mean temperature 2009')
plt.legend()
plt.savefig('Figures/fourier4.png', dpi=600)